In [145]:
# Package imports
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

## Generating a dataset

Let's start by generating a dataset we can play with. Fortunately, [scikit-learn](http://scikit-learn.org/) has some useful dataset generators, so we don't need to write the code ourselves. We will go with the [`make_moons`](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) function.

In [146]:
# Generate a dataset and plot it
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',  categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',  categories=categories)

print(newsgroups_train.data[0])

From: rych@festival.ed.ac.uk (R Hawkes)
Subject: 3DS: Where did all the texture rules go?
Lines: 21

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Rycharde Hawkes				email: rych@festival.ed.ac.uk
Virtual Environment Laboratory
Dept. of Psychology			Tel  : +44 31 650 3426
Univ. of Edinburgh			Fax  : +44 31 667 0150



In [212]:
num_train = len(newsgroups_train.data)
num_test  = len(newsgroups_test.data)

vectorizer = TfidfVectorizer(max_features=80)

X = vectorizer.fit_transform( newsgroups_train.data + newsgroups_test.data )
X_train = X[0:num_train, :]
X_test = X[num_train:num_train+num_test,:]

Y_train = newsgroups_train.target
Y_test = newsgroups_test.target

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(2034, 80) (2034,)
(1353, 80) (1353,)


In [148]:
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [213]:
# Helper function to evaluate the total loss on the dataset
def calculate_loss(model,X, y):
    W1, b1, W2, b2, W3, b3, W4, b4, W5, b5 = model['W1'], model['b1'], model['W2'], model['b2'], model['W3'], model['b3'], model['W4'], model['b4'], model['W5'], model['b5']
    # Forward propagation to calculate our predictions
    z1 = X.dot(W1) + b1
    a1 = (abs(z1) + z1) / 2
    z2 = a1.dot(W2) + b2
    a2 = (abs(z2) + z2) / 2
    z3 = a2.dot(W3) + b3
    a3 = (abs(z3) + z3) / 2
    z4 = a3.dot(W4) + b4
    a4 = (abs(z4) + z4) / 2
    z5 = a4.dot(W5) + b5
    #exp_scores = np.exp(z2)
    exp_scores = np.exp(z5)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # Calculating the loss
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    # Add regulatization term to loss (optional)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)) + np.sum(np.square(W4)) + np.sum(np.square(W5)))
    return 1./num_examples * data_loss

We also implement a helper function to calculate the output of the network. It does forward propagation as defined above and returns the class with the highest probability.

In [164]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2, W3, b3, W4, b4, W5, b5 = model['W1'], model['b1'], model['W2'], model['b2'], model['W3'], model['b3'], model['W4'], model['b4'], model['W5'], model['b5']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = (abs(z1) + z1) / 2
    z2 = a1.dot(W2) + b2
    a2 = (abs(z2) + z2) / 2
    z3 = a2.dot(W3) + b3
    a3 = (abs(z3) + z3) / 2
    z4 = a3.dot(W4) + b4
    a4 = (abs(z4) + z4) / 2
    z5 = a4.dot(W5) + b5
    #exp_scores = np.exp(z2)
    exp_scores = np.exp(z5)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

Finally, here comes the function to train our Neural Network. It implements batch gradient descent using the backpropagation derivates we found above.

In [214]:
# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(X, y, nn_hdim, epsilon=0.01, reg_lambda=0, num_passes=20000,  print_loss=False):
    
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    nn_input_dim=80
    nn_output_dim=4
    W1 = np.random.randn(nn_input_dim, nn_hdim[0]) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim[0]))
    W2 = np.random.randn(nn_hdim[0], nn_hdim[1]) / np.sqrt(nn_hdim[0])
    b2 = np.zeros((1, nn_hdim[1]))
    W3 = np.random.randn(nn_hdim[1], nn_hdim[2]) / np.sqrt(nn_hdim[1])
    b3 = np.zeros((1, nn_hdim[2]))
    W4 = np.random.randn(nn_hdim[2], nn_hdim[3]) / np.sqrt(nn_hdim[2])
    b4 = np.zeros((1, nn_hdim[3]))
    W5 = np.random.randn(nn_hdim[3], nn_output_dim) / np.sqrt(nn_hdim[3])
    b5 = np.zeros((1, nn_output_dim))
    # This is what we return at the end
    model = {}
    
    # Gradient descent. For each batch...
    for i in range(0, num_passes):

        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = (abs(z1) + z1) / 2
        z2 = a1.dot(W2) + b2
        a2 = (abs(z2) + z2) / 2
        z3 = a2.dot(W3) + b3
        a3 = (abs(z3) + z3) / 2
        z4 = a3.dot(W4) + b4
        a4 = (abs(z4) + z4) / 2
        z5 = a4.dot(W5) + b5
        #exp_scores = np.exp(z2)
        exp_scores = np.exp(z5)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        #print(z5)
        # Backpropagation
        delta5 = probs
        #print(probs)
        delta5[range(num_examples), y] -= 1
        #print(probs)
        dW5 = (a4.T).dot(delta5)
        #print(dW5)
        db5 = np.sum(delta5, axis=0, keepdims=True)
        #print(db5)
        #delta2 = delta3.dot(W3.T) * (1 - np.power(a2, 2))
        
        tempRelu = a4.copy()
        tempRelu[tempRelu > 0] = 1
        delta4 = delta5.dot(W5.T) * tempRelu
        dW4 = (a3.T).dot(delta4)
        db4 = np.sum(delta4, axis=0, keepdims=True)
        
        tempRelu = a3.copy()
        tempRelu[tempRelu > 0] = 1
        delta3 = delta4.dot(W4.T) * tempRelu
        dW3 = (a2.T).dot(delta3)
        db3 = np.sum(delta3, axis=0, keepdims=True)
        
        tempRelu = a2.copy()
        tempRelu[tempRelu > 0] = 1
        delta2 = delta3.dot(W3.T) * tempRelu
        dW2 = (a1.T).dot(delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        
        tempRelu = a1.copy()
        tempRelu[tempRelu > 0] = 1
        #print(tempRelu)
        delta = delta2.dot(W2.T) * tempRelu
        #delta = delta2.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = (X.T).dot(delta)
        db1 = np.sum(delta, axis=0)
        
        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW5 += reg_lambda * W5
        dW4 += reg_lambda * W4
        dW3 += reg_lambda * W3
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1
        
        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        W3 += -epsilon * dW3
        b3 += -epsilon * db3
        W4 += -epsilon * dW4
        b4 += -epsilon * db4
        W5 += -epsilon * dW5
        b5 += -epsilon * db5
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3, 'W4': W4, 'b4': b4, 'W5': W5, 'b5': b5}
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
            print("Loss after iteration %i: %f" %(i, calculate_loss(model,X,y)))
    
    return model

### A network with a hidden layer of size 3

Let's see what happens if we train a network with a hidden layer size of 3.


In [216]:
# Build a model with a 3-dimensional hidden layer
num_examples, input_dim = X_train.shape
epsilon = 0.0001
reg_lambda = 0.01
epochs = 5000

model = build_model(X_train, Y_train, [32,32,32,32], epsilon, reg_lambda, epochs, print_loss=True)

Loss after iteration 0: 1.383305
Loss after iteration 1000: 0.667686
Loss after iteration 2000: 0.434912
Loss after iteration 3000: 0.221577
Loss after iteration 4000: 0.059148


Yay! This looks pretty good. Our neural networks was able to find a decision boundary that successfully separates the classes.

# Varying the hidden layer size

In the example above we picked a hidden layer size of 3. Let's now get a sense of how varying the hidden layer size affects the result.


In [217]:
n_correct = 0
n_test = X_test.shape[0]
for n in range(n_test):
    x = X_test[n,:]
    yp = predict(model, x)
    if yp == Y_test[n]:
        n_correct += 1.0

print('Accuracy %f = %d / %d'%(n_correct/n_test, int(n_correct), n_test) )

Accuracy 0.598670 = 810 / 1353


We can see that while a hidden layer of low dimensionality nicely capture the general trend of our data, but higher dimensionalities are prone to overfitting. They are "memorizing" the data as opposed to fitting the general shape. If we were to evaluate our model on a separate test set (and you should!) the model with a smaller hidden layer size would likely perform better because it generalizes better. We could counteract overfitting with stronger regularization, but picking the correct size for hidden layer is a much more "economical" solution.

# Exercises

Here are some things you can try to become more familiar with the code:

1. Instead of batch gradient descent, use minibatch gradient descent ([more info](http://cs231n.github.io/optimization-1/#gd)) to train the network. Minibatch gradient descent typically performs better in practice. 
2. We used a fixed learning rate $\epsilon$ for gradient descent. Implement an annealing schedule for the gradient descent learning rate ([more info](http://cs231n.github.io/neural-networks-3/#anneal)). 
3. We used a $\tanh$ activation function for our hidden layer. Experiment with other activation functions (some are mentioned above). Note that changing the activation function also means changing the backpropagation derivative.
4. Extend the network from two to three classes. You will need to generate an appropriate dataset for this.
5. Extend the network to four layers. Experiment with the layer size. Adding another hidden layer means you will need to adjust both the forward propagation as well as the backpropagation code.
